In [1]:
import pandas as pd

In [2]:
link1 = "./BD/df_basic_name_clean.csv"
link2 = "./BD/df_t_akas_clean.csv"
link3 = "./BD/df_t_basics_clean.csv"
link4 = "./BD/df_t_crew_clean.csv"
link5 = "./BD/df_t_episode_clean.csv"
link6 = "./BD/df_t_principals_clean.csv"
link7 = "./BD/df_t_ratings_clean.csv"
link8 = "./BD/tmdb_full.csv"
link9 = "./BD/df_tID.csv"

In [4]:
df_basic_name = pd.read_csv(link1)

In [5]:
df_t_akas = pd.read_csv(link2)

In [ ]:
df_t_basics = pd.read_csv(link3)

In [8]:
df_t_crew = pd.read_csv(link4)

In [ ]:
df_t_episode = pd.read_csv(link5nope)

In [63]:
df_t_principals = pd.read_csv(link6)

KeyboardInterrupt: 

In [10]:
df_t_ratings = pd.read_csv(link7)

In [14]:
df_tmdb = pd.read_csv(link8)

C:\Users\Antoine\AppData\Local\Temp\ipykernel_14296\4261131250.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv(link8)


In [58]:
df_tID = pd.read_csv(link9)

On veut 2 df: un df avec index tconst et un avec index nconst
1.  Commençons par df_tID

In [ ]:
# Prépartion table title basics en premier car elle permet de selectionner les 'movies' et les années souhaitées
df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie') & (df_t_basics['startYear'] >= 1950) & (df_t_basics['isAdult'] == 0)]
df_tb_tconst = df_tb_tconst[['tconst', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']]

#Préparation table title principals
df_ptconst = df_t_principals.groupby('tconst').agg(list).reset_index()
df_ptconst = df_ptconst.drop('category', axis = 1)

#Premier merge avec title basic en pivot
df_tID = pd.merge(df_tb_tconst, df_ptconst, how = 'left', left_on='tconst', right_on='tID')

#2e merge avec akas deja nettoyé
df_tID = pd.merge(df_tID, df_t_akas, how = 'left', left_on='tconst', right_on='titleId')

#3e merge avec ratings déjà nettoyé
df_tID = pd.merge(df_tID, df_t_ratings, how = 'left', on='tconst')

#Renommage des colonnes
df_tID = df_tID.rename(columns={'tconst' : 't_ID', 'primaryTitle' : 'Titre_Principal', 'startYear' : 'Année', 'genres' : 'Genres', 'runtimeMinutes' : 'Durée', 'nID_implique' : 'n_ID_liste', 'title' : 'Titres_possibles' 'averageRating' : 'Note_moyenne', 'numVotes' : 'Nb_votants'})

#Drop colonnes doublons
df_tID = df_tID.drop(['tID', 'titleId'], axis = 1)

#Trie par année
df_tID = df_tID.sort_values('Année')

#reset index
df_tID= df_tID.reset_index().drop('index', axis = 1)

#Export
df_tID.to_csv('df_tID.csv', index = False)

Pourquoi pas un dropna qui retire la moitié du df mais nous permet de travailler sur un propre avec uniquement des valeurs renseignées...

In [59]:
df_tID

,index,t_ID,Titre_Principal,Année,Durée,Genres,n_ID_liste,Titres_possibles,Note_moyenne,Nb_votants
0,251279,tt1623633,Aur Bhi Gham Hain,1960.0,NaN,['Drama'],"['nm1524409', 'nm0713607', 'nm0750722', 'nm350...","['Aur Bhi Gham Hain', 'Aur Bhi Gham Hain']",NaN,NaN
1,275372,tt1935323,Zassô no yô na inochi,1960.0,91.0,['\\N'],"['nm0038377', 'nm0368969', 'nm0408354', 'nm044...","['Zassô no yô na inochi', '雑草のような命']",NaN,NaN
2,490573,tt8948610,All These People,1960.0,NaN,['Documentary'],"['nm0201438', 'nm0236111', 'nm0450591', 'nm158...","['All These People', 'All These People']",NaN,NaN
3,275272,tt1934375,Roku-san-sei gurentai,1960.0,85.0,['\\N'],"['nm1011131', 'nm2576430', 'nm2764570', 'nm446...","['Roku-san-sei gurentai', '六三制愚連隊']",NaN,NaN
4,490321,tt8933624,Ore wa ginza no ki e itai,1960.0,78.0,['Action'],"['nm0905268', 'nm0793675', 'nm4440521', 'nm232...","['Ore wa ginza no ki e itai', 'I Am a Ginza Ca...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
502555,323145,tt26217472,Rajju,2029.0,115.0,"['Adventure', 'Fantasy', 'Sci-Fi']","['nm10266613', 'nm0219266']","['Rajju', 'Rajju']",NaN,NaN
502556,372104,tt32293987,The Time Machine,2029.0,NaN,['Sci-Fi'],"['nm4224794', 'nm8200022', 'nm8817238', 'nm133...","['The Time Machine', 'The Time Machine']",NaN,NaN
502557,360909,tt3095356,Avatar 4,2029.0,NaN,"['Action', 'Adventure', 'Fantasy']","['nm0757855', 'nm0000706', 'nm0000244', 'nm131...","['Avatar 4', 'Avatar 4', 'Avatar 4', 'Avatar 4...",NaN,NaN
502558,423153,tt4985164,Hurry Scurry,2030.0,NaN,['\\N'],"['nm0625337', 'nm0737137', 'nm9407666', 'nm940...","['Hurry Scurry', 'Hurry Scurry', 'Hurry Scurry...",NaN,NaN


In [62]:
df_tID[(df_tID['Année'] < 1980)]

,index,t_ID,Titre_Principal,Année,Durée,Genres,n_ID_liste,Titres_possibles,Note_moyenne,Nb_votants
0,251279,tt1623633,Aur Bhi Gham Hain,1960.0,NaN,['Drama'],"['nm1524409', 'nm0713607', 'nm0750722', 'nm350...","['Aur Bhi Gham Hain', 'Aur Bhi Gham Hain']",NaN,NaN
1,275372,tt1935323,Zassô no yô na inochi,1960.0,91.0,['\\N'],"['nm0038377', 'nm0368969', 'nm0408354', 'nm044...","['Zassô no yô na inochi', '雑草のような命']",NaN,NaN
2,490573,tt8948610,All These People,1960.0,NaN,['Documentary'],"['nm0201438', 'nm0236111', 'nm0450591', 'nm158...","['All These People', 'All These People']",NaN,NaN
3,275272,tt1934375,Roku-san-sei gurentai,1960.0,85.0,['\\N'],"['nm1011131', 'nm2576430', 'nm2764570', 'nm446...","['Roku-san-sei gurentai', '六三制愚連隊']",NaN,NaN
4,490321,tt8933624,Ore wa ginza no ki e itai,1960.0,78.0,['Action'],"['nm0905268', 'nm0793675', 'nm4440521', 'nm232...","['Ore wa ginza no ki e itai', 'I Am a Ginza Ca...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
71560,172204,tt11193856,Dehshat,1979.0,NaN,"['Action', 'Crime']","['nm0019455', 'nm0706691', 'nm1531125', 'nm380...","['Dehshat', 'Dehshat']",NaN,NaN
71561,19021,tt0079832,Running,1979.0,102.0,"['Drama', 'Sport']","['nm0000140', 'nm0000769', 'nm0199325', 'nm050...","['Running', 'Running - Der Moment seines Leben...",5.9,846.0
71562,382583,tt33319911,Ibadat,1979.0,NaN,['Drama'],"['nm0019455', 'nm9866129', 'nm3039108', 'nm376...","['Ibadat', 'Ibadat']",NaN,NaN
71563,79404,tt0235878,What the Hell's Going on Up There?,1979.0,28.0,"['Animation', 'Documentary']","['nm0250616', 'nm0482943', 'nm0482943', 'nm008...","[""What the Hell's Going on Up There?"", ""What t...",8.3,12.0


2. Ensuite un df_nID

In [ ]:
#Tri de la table title basic pour avoir les tconst correspondant aux films
df_tb_nconst = df_t_basics[df_t_basics['titleType'] == 'movie']
df_tb_nconst = df_tb_nconst[['tconst']]
df_tb_nconst = df_tb_nconst.reset_index()

# Préparation table title principals avec seulement les acteurs
df_t_principals_fusion = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_fusion = df_t_principals_fusion.drop('category', axis = 1)

#Premier groupby tconst pour merge avec les tconst triés de title basics
df_t_principals_fusion.groupby('tconst').agg(list).reset_index()

#Premier merge par tconst
df_nID = pd.merge(df_t_principals_fusion, df_tb_nconst, how = 'left', on = 'tconst')

In [34]:
df_nID

,tconst,nconst
0,tt0000005,nm0443482
1,tt0000005,nm0653042
2,tt0000007,nm0179163
3,tt0000007,nm0183947
4,tt0000008,nm0653028
...,...,...
36010016,tt9916880,nm0254176
36010017,tt9916880,nm0286175
36010018,tt9916880,nm1052583
36010019,tt9916880,nm2676923


In [20]:
df_t_principals_fusion['Nb_films'] = df_t_principals_fusion['tconst'].apply(lambda x : len(x))

In [22]:
df_t_principals_fusion.sort_values('Nb_films')

,nconst,tconst,Nb_films
1716120,nm1882764,[tt0455481],1
1675197,nm1737388,[tt2292180],1
3008743,nm7858936,[tt5350300],1
3008742,nm7858935,[tt5350300],1
1675200,nm1737398,[tt11758088],1
...,...,...,...
394526,nm10120013,"[tt10626454, tt10630672, tt10630674, tt1063067...",9905
1953471,nm2900549,"[tt0344642, tt18270646, tt18374846, tt18374848...",9946
309389,nm0815658,"[tt0189130, tt0344642, tt0350053, tt0360385, t...",9996
992467,nm1296472,"[tt0189130, tt0336227, tt0336716, tt0344642, t...",10646


In [15]:
df_tmdb_clean = df_tmdb[['imdb_id', 'budget', 'backdrop_path',  'homepage', 'overview', 'poster_path', 'popularity', 'tagline', 'vote_average', 'vote_count']]

In [16]:
df_tmdb_clean

,imdb_id,budget,backdrop_path,homepage,overview,poster_path,popularity,tagline,vote_average,vote_count
0,tt0029927,0,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,NaN,Blondie and Dagwood are about to celebrate the...,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,2.852,The favorite comic strip of millions at last o...,7.214,7
1,tt0011436,0,NaN,NaN,NaN,/6xUbUCvndklbGVYiljHr34NTxSl.jpg,1.091,NaN,0.000,0
2,tt0055747,0,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,NaN,Love at Twenty unites five directors from five...,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,3.770,The Intimate Secrets of Young Lovers,6.700,41
3,tt0094675,0,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,Taisto Kasurinen is a Finnish coal miner whose...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,9.214,NaN,7.046,248
4,tt0092149,0,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,"An episode in the life of Nikander, a garbage ...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg,6.282,NaN,7.182,269
...,...,...,...,...,...,...,...,...,...,...
309567,tt24962414,0,/oGW8uF821i8VowzpmECbxk7QXNs.jpg,NaN,The film tells the story of the heavy-duty pol...,/tzF2BndyY3HIf2KjJR1FBYoIqTp.jpg,1.307,NaN,0.000,0
309568,tt17352384,0,/oQ429AcD85ttxvOxAaYpETnAsW0.jpg,NaN,Mozambique requests from Russia is being helpe...,/nqMk6TIUwqSMRc6QPKYguNr9Ql0.jpg,1.238,NaN,5.000,3
309569,tt8223104,0,/fJhruCWu4sx3TKhFigEVntiVL93.jpg,https://selectedfilms.com/marcelino-el-mejor-p...,"An account of the life, work and many triumphs...",/btNlb2H3NXuBCN5J1FB9IVddYR.jpg,0.600,NaN,6.000,2
309570,tt14213656,0,NaN,NaN,"Three Canadian Holocaust survivors, with unans...",/uLfathKx9hauVKdasJc8A720kvb.jpg,2.558,"Three Canadian Holocaust survivors, with unans...",0.000,0


In [ ]:
df_tID = pd.merge(df_tID, df_tmdb_tconst, how = 'left', left_on= 't_ID', right_on = 'imdb_id')

In [91]:
df_tID = df_tID.drop('imdb_id', axis = 1).reset_index()